# Aмангелді Нұрғалым - Зертханалық жұмыс

## Представление данных

In [1]:
# Один обучающий/тестовый пример для классификации последовательностей
class InputExample(object):
    # Создает пример ввода.
    def __init__(self, guid, text_a, text_b=None, labels=None):
        self.guid = guid # Создает пример ввода.
        self.text_a = text_a #Неотмеченный текст первой последовательности. 
        self.text_b = text_b # Неотмеченный текст второй последовательности.
        self.labels = labels # Метка примера. Это должно быть указано для примеров train и dev, но не для тестовых примеров.

### Этот код определяет класс InputFeatures, который служит для представления набора свойств (features) для обработки входных данных моделью машинного обучения, такой как модель BERT.

In [2]:
# Единый набор свойств данных
class InputFeatures(object):
    def __init__(self, input_ids, input_mask, segment_ids, label_ids):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_ids = label_ids

## Архитектура модели

## Модель BERT для классификации

In [12]:
# Этот модуль состоит из наилучшей модели с линейным слоем поверх объединенных выходных данных.
# Импорт необходимых библиотек и классов
from transformers.modeling_utils import PreTrainedModel
import torch

# Определение класса модели BERT для многозначной классификации последовательностей,
# который наследует от PreTrainedModel из transformers
class BertForMultiLabelSequenceClassification(PreTrainedModel):
    def __init__(self, config, num_labels=2):
        # Инициализация класса с использованием конфигурации и указанием числа меток
        super(BertForMultiLabelSequenceClassification, self).__init__(config)
        
        # Задание числа меток
        self.num_labels = num_labels
        
        # Создание экземпляра модели BERT с использованием переданной конфигурации
        self.bert = BertModel(config)
        
        # Добавление слоя dropout для предотвращения переобучения
        self.dropout = torch.nn.Dropout(config.hidden_dropout_prob)
        
        # Линейный слой для классификации с выходной размерностью, равной числу меток
        self.classifier = torch.nn.Linear(config.hidden_size, num_labels)
        
        # Применение метода для инициализации весов BERT
        self.apply(self.init_bert_weights)

    # Определение forward-метода для прямого прохода модели
    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
        # Получение выходных данных от BERT
        _, pooled_output = self.bert(input_ids, token_type_ids, attention_mask, output_all_encoded_layers=False)
        
        # Применение dropout к выходу BERT
        pooled_output = self.dropout(pooled_output)
        
        # Подача выхода через линейный классификатор
        logits = self.classifier(pooled_output)

        # Если есть метки, вычисление функции потерь
        if labels is not None:
            loss_fct = BCEWithLogitsLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1, self.num_labels))
            return loss
        else:
            # В противном случае возвращение выхода модели
            return logits
        
    # Метод для замораживания параметров BERT
    def freeze_bert_encoder(self):
        for param in self.bert.parameters():
            param.requires_grad = False
    
    # Метод для размораживания параметров BERT
    def unfreeze_bert_encoder(self):
        for param in self.bert.parameters():
            param.requires_grad = True

#### Основное изменение здесь — использование бинарной кросс-энтропийной функции потерь с logit-функцией (BCEWithLogitsLoss) вместо классической кросс-энтропийной функции, используемой для мультиклассовой классификации. Бинарная кросс-энтропийная потеря позволяет нашей модели присваивать меткам независимые вероятности.

## Код модели показывает слои модели вместе с их размерами.

In [ ]:
from transformers import BertConfig, BertForSequenceClassification
from transformers.models.bert.modeling_bert import BertEmbeddings, BertEncoder, BertLayer, BertAttention, BertSelfAttention, BertSelfOutput, BertIntermediate, BertOutput, BertPooler
from torch.nn import Embedding, Linear, ModuleList, FusedLayerNorm, Dropout, Tanh
# Определение модели BERT для многозначной классификации последовательностей
model = BertForSequenceClassification(
    config=BertConfig(
        # Встраивание токенов
        embeddings=BertEmbeddings(
            # Встраивание слов
            word_embeddings=Embedding(28996, 768),
            # Встраивание позиций токенов
            position_embeddings=Embedding(512, 768),
            # Встраивание типов токенов
            token_type_embeddings=Embedding(2, 768),
            # Нормализация слоев
            LayerNorm=FusedLayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True),
            # Dropout для предотвращения переобучения
            dropout=Dropout(p=0.1)
        ),
        # Слои энкодера BERT
        encoder=BertEncoder(
            # Список слоев BERT
            layer=ModuleList(
                # Один из 12 слоев BERT
                BertLayer(
                    # Механизм внимания внутри слоя
                    attention=BertAttention(
                        # Собственно внимание
                        self=BertSelfAttention(
                            # Линейные трансформации для запроса, ключа и значения
                            query=Linear(in_features=768, out_features=768, bias=True),
                            key=Linear(in_features=768, out_features=768, bias=True),
                            value=Linear(in_features=768, out_features=768, bias=True),
                            # Dropout для предотвращения переобучения
                            dropout=Dropout(p=0.1)
                        ),
                        # Выход из механизма внимания
                        output=BertSelfOutput(
                            # Линейный слой для выхода
                            dense=Linear(in_features=768, out_features=768, bias=True),
                            # Нормализация слоев
                            LayerNorm=FusedLayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True),
                            # Dropout для предотвращения переобучения
                            dropout=Dropout(p=0.1)
                        )
                    ),
                    # Промежуточный слой с полносвязным перцептроном
                    intermediate=BertIntermediate(
                        dense=Linear(in_features=768, out_features=3072, bias=True)
                    ),
                    # Выходной слой слоя BERT
                    output=BertOutput(
                        # Линейный слой для выхода
                        dense=Linear(in_features=3072, out_features=768, bias=True),
                        # Нормализация слоев
                        LayerNorm=FusedLayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True),
                        # Dropout для предотвращения переобучения
                        dropout=Dropout(p=0.1)
                    )
                )
            )
        ),
        # Слой пулинга для создания представления последовательности
        pooler=BertPooler(
            # Линейный слой для пулинга
            dense=Linear(in_features=768, out_features=768, bias=True),
            # Функция активации Tanh
            activation=Tanh()
        )
    ),
    # Dropout для предотвращения переобучения
    dropout=Dropout(p=0.1),
    # Линейный слой для классификации с заданным числом классов
    classifier=Linear(in_features=768, out_features=6, bias=True)
)

# Метрика качества

In [ ]:
def accuracy_thresh(y_pred:Tensor, y_true:Tensor, thresh:float=0.5, sigmoid:bool=True):
 # "Вычислите точность, когда `y_pred` и `y_true` имеют одинаковый размер".
    if sigmoid: y_pred = y_pred.sigmoid()

    return np.mean(((y_pred>thresh)==y_true.byte()).float().cpu().numpy(), axis=1).sum()

In [ ]:
from sklearn.metrics import roc_curve, auc

# Вычислить кривую ROC и площадь ROC для каждого класса
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(num_labels):
    fpr[i], tpr[i], _ = roc_curve(all_labels[:, i], all_logits[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# # Компьютерная микро-усредненная кривая ROC и площадь ROC
fpr["micro"], tpr["micro"], _ = roc_curve(all_labels.ravel(), all_logits.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])